In [1]:
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, losses
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import callbacks
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda, Cropping2D

import pandas as pd
from keras.utils import np_utils

from PIL import Image
import numpy as np

Using TensorFlow backend.


In [2]:
data = pd.read_csv('train_concepts.txt', delimiter=":", header=None)

In [3]:
data.head()

,0,1
0,C0738276,btt-8-083Fig3;abcd-28-01-0087-g03
1,C1287716,ipej030157-10
2,C0576088,aps-43-590-g001;aps-43-590-g002;aps-43-590-g00...
3,C2332719,CRIRH2016-2019250.001
4,C0796089,kjlm-31-49-g001;MGG3-4-599-g001;JPN-6-19-g004;...


In [4]:
res = []
for index, row in data[:30].iterrows():
#     print index
    for i in row[1].split(';'):
#         print str(i)+';'+str(row[0])+';'+str(index)
        res.append(str(i)+';'+str(row[0])+';'+str(index))        

In [5]:
final_results = pd.DataFrame(
    {
        "0":res,
    })
print final_results.head()

                                0
0        btt-8-083Fig3;C0738276;0
1  abcd-28-01-0087-g03;C0738276;0
2        ipej030157-10;C1287716;1
3      aps-43-590-g001;C0576088;2
4      aps-43-590-g002;C0576088;2


In [6]:
final_results.to_csv('Final_prediction1000.csv',  index = False, header=False)

In [7]:
now = pd.read_csv('/Users/sharath/Desktop/GP/sharath/Final_prediction1000.csv', delimiter=";", header=None)
k = now[[2]].values
res = np_utils.to_categorical(k)
# print res[:5]
leng =  res.shape[1]
print len(now[0]), len(res)

dataset_size  = len(now[0])

# print now.head()

labels = pd.DataFrame(res)
# print labels.head()


result = pd.concat([now, labels], axis=1, join_axes=[now.index])
# print result.head()
# result.to_csv("preprocessed.txt", header=False)


1424 1424


In [8]:
imgdim = 80

In [9]:


model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape = (imgdim, imgdim,3),border_mode = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())
model.add(Dense(500, activation = 'relu'))
model.add(Dense(leng, activation = 'softmax')) 


# model = Sequential()
# model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(64,64,3)))
# model.add(Conv2D(32, (5, 5), activation='relu', padding='same'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
# model.add(Flatten())
# # model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()


/Users/sharath/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu", input_shape=(80, 80, 3...)`
  after removing the cwd from sys.path.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 80, 80, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 40, 40, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               25600500  
_________________________________________________________________
dense_2 (Dense)              (None, 30)                15030     
Total params: 25,616,426
Trainable params: 25,616,426
Non-trainable params: 0
_________________________________________________________________


In [43]:
def BatchGenerator(track_index):
    track_index = track_index
    X = []
    y = []
    print  track_index
    for index, row in result[track_index:track_index+plus_track_index].iterrows():
#         print row.index
        x = row.values[0]
        label = row.values[3:]
        x = row.values[0]
        y.append(label)
        img = Image.open("/Users/sharath/Desktop/GP/CaptionTraining2018/"+ x +".jpg")
        img = img.resize((imgdim,imgdim)).convert('RGB')
        T = (np.array(img))
        X.append(T)
#         track_index =  index
    X_train = np.array(X)
    y_train = np.array(y)
#     print X_train.shape, y_train.shape
    return X_train, y_train
    

nb_epoch = 1
track_index = 0
# plus_track_index = (dataset_size/10)
plus_track_index = 1000

for e in range(nb_epoch):
    print("epoch %d" % e)
    X,y = BatchGenerator(track_index) 
    print X.shape,y.shape
    model.fit(X, y)
    track_index = track_index + plus_track_index
        


epoch 0
0
(1000, 64, 64, 3) (1000, 30)
Epoch 1/1
1000/1000 [==============================] - 12s 12ms/step - loss: 13.5217 - acc: 0.1610


In [ ]:
# datagen = ImageDataGenerator(
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True)

# # compute quantities required for featurewise normalization
# # (std, mean, and principal components if ZCA whitening is applied)
# datagen.fit(x_train)

# # # fits the model on batches with real-time data augmentation:
# # model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
# #                     steps_per_epoch=len(x_train) / 32, epochs=epochs)

# # here's a more "manual" example
# for e in range(epochs):
#     print('Epoch', e)
#     batches = 0
#     for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=32):
#         model.fit(x_batch, y_batch)
#         batches += 1
#         if batches >= len(x_train) / 32:
#             # we need to break the loop by hand because
#             # the generator loops indefinitely
#             break


In [ ]:
 # def generate_arrays_from_file():
    
#     while 1:
#         f = open(path)
#         for line in f:
#             # create numpy arrays of input data
#             # and labels, from each line in the file
#             x, y = process_line(line)
#             img = load_images(x)
#             yield (img, y)
#         f.close()

        
# model.fit_generator(generate_arrays_from_file(),steps_per_epoch='10',epochs=10)


# log_dir = './tf-log/'
# tb_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# cbks = [tb_cb]

# target_dir = './models/'
# if not os.path.exists(target_dir):
#   os.mkdir(target_dir)
# model.save('./models/model.h5')
# model.save_weights('./models/weights.h5')

In [1]:


# def generate_arrays_from_file():
#     for index, row in now.iterrows():
#         print index
#         x = row.values[0]
#         y = res[index]
#         img = Image.open("/Users/sharath/Desktop/GP/CaptionTraining2018/"+ x +".jpg")
#         T.append(np.array(img))
#         X = np.array(T)
#     yield (X, y)
    
# model.fit_generator(generate_arrays_from_file(),steps_per_epoch='1',epochs=1)






# log_dir = './tf-log/'
# tb_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# cbks = [tb_cb]

# target_dir = './models/'
# if not os.path.exists(target_dir):
#   os.mkdir(target_dir)
# model.save('./models/model.h5')
# model.save_weights('./models/weights.h5')